In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col,max,count,sum,mean,stddev_pop,hour,countDistinct,expr,stddev,window,column
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
import math

In [2]:
# sc.stop()

# Initialization of SparkConf which is required for Spark Context
conf = SparkConf().setAppName('myapp').setMaster('local')

# Initialization of SparkContext
sc = SparkContext().getOrCreate(conf=conf)
sc.setLogLevel("OFF")

# Initialization of SparkSession into spark variable
spark = SparkSession(sc)

# Assignment 3

In [3]:
# Displaying Dataset
dataframe = spark.read.csv('retail_data.csv', header=True, inferSchema=True)
dataframe.createTempView('retail_data')
dataframe.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

### How many orders did customers perform at which hour? (SQL)

In [21]:
spark.sql("select HOUR(InvoiceDate) AS HOUR, COUNT(DISTINCT(InvoiceNo)) AS Orders from retail_data group by HOUR(InvoiceDate)").show()
dataframe.select("InvoiceNo", hour("InvoiceDate").alias("HOUR")).groupBy("HOUR").agg(countDistinct("InvoiceNo")).alias("Orders").show()

# dataframe.select("InvoiceNo", hour("InvoiceDate").alias("HOUR")).groupBy("InvoiceNo","HOUR").agg(countDistinct("InvoiceNo")).alias("Orders").show()
# spark.sql("select StockCode,HOUR(InvoiceDate) AS HOUR, COUNT(DISTINCT(InvoiceNo)) AS Orders from retail_data group by StockCode,HOUR(InvoiceDate)").show()
# spark.sql("select CustomerID,InvoiceNo, COUNT(InvoiceNo) AS Orders from retail_data where CustomerId is not null group by CustomerID,InvoiceNo").show()

+----+------+
|HOUR|Orders|
+----+------+
|  12|    23|
|  13|    12|
|  16|    17|
|  15|    15|
|   9|    18|
|  17|     6|
|   8|     6|
|  10|    12|
|  11|    13|
|  14|    21|
+----+------+

+----+----------------+
|HOUR|count(InvoiceNo)|
+----+----------------+
|  12|              23|
|  13|              12|
|  16|              17|
|  15|              15|
|   9|              18|
|  17|               6|
|   8|               6|
|  10|              12|
|  11|              13|
|  14|              21|
+----+----------------+



### How frequently was each product bought in different countries? (SQL)

In [22]:
 spark.sql("select Country, StockCode, Count(*) as Count from retail_data group by StockCode, Country").show()

+--------------+---------+-----+
|       Country|StockCode|Count|
+--------------+---------+-----+
|United Kingdom|    22154|    1|
|United Kingdom|    22478|    1|
|United Kingdom|    22844|    3|
|United Kingdom|    22768|    4|
|United Kingdom|    16238|    3|
|United Kingdom|    22916|    3|
|United Kingdom|    21704|    2|
|United Kingdom|        M|    2|
|United Kingdom|   35271S|    1|
|United Kingdom|    70007|    4|
|       Germany|    22242|    2|
|United Kingdom|    72741|    4|
|United Kingdom|   84534B|    1|
|United Kingdom|    22342|    1|
|United Kingdom|   46776B|    1|
|United Kingdom|    21816|    1|
|United Kingdom|    22375|    1|
|United Kingdom|   85232B|    5|
|United Kingdom|   84596E|    1|
|United Kingdom|    21851|    2|
+--------------+---------+-----+
only showing top 20 rows



### How many orders did customers perform at which hour? (Dataframe)

In [23]:
dataframe.select(hour('InvoiceDate').alias('HOUR'), 'InvoiceNo').groupby('HOUR').agg(countDistinct('InvoiceNo')).show()

+----+----------------+
|HOUR|count(InvoiceNo)|
+----+----------------+
|  12|              23|
|  13|              12|
|  16|              17|
|  15|              15|
|   9|              18|
|  17|               6|
|   8|               6|
|  10|              12|
|  11|              13|
|  14|              21|
+----+----------------+



### How frequently was each product bought in different countries? (Dataframe)

In [24]:
#Variant one
dataframe.show()
# dataframe.select('Country', 'StockCode', 'Quantity').groupby('Country', 'StockCode').agg(sum('Quantity').alias('Quantity')).show()
dataframe.select('Country', 'StockCode', 'Quantity').groupby('Country', 'StockCode').sum('Quantity')\
.withColumnRenamed('sum(Quantity)','Quantity').show()
# dataframe.select('Country', 'StockCode', 'Quantity').groupby('Country', 'StockCode').agg(count('Quantity').alias('Quantity')).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [25]:
# Variant two
dataframe.groupby('Country', 'StockCode').agg(sum('Quantity').alias('Quantity')).show()

+--------------+---------+--------+
|       Country|StockCode|Quantity|
+--------------+---------+--------+
|United Kingdom|    21125|      12|
|United Kingdom|    21794|       1|
|United Kingdom|    21662|       3|
|          EIRE|   85136C|       2|
|United Kingdom|   84032A|      22|
|United Kingdom|   84086C|       2|
|United Kingdom|   90214H|       2|
|United Kingdom|    22753|       9|
|United Kingdom|    21557|       3|
|United Kingdom|    21743|      15|
|United Kingdom|    22488|      12|
|United Kingdom|    22956|       4|
|        Norway|    22531|      24|
|United Kingdom|    22728|      13|
|United Kingdom|    82551|       3|
|United Kingdom|    22581|       4|
|United Kingdom|    21671|       2|
|United Kingdom|   90177E|       1|
|United Kingdom|   84926A|       4|
|United Kingdom|    20726|       2|
+--------------+---------+--------+
only showing top 20 rows



### Select from a datacube a slice that contains only items from the United States.

In [26]:
# For slice operation, use where clause on only one column and NOT MORE
dataframe.where(col("Country") == "United States").show()
# dataframe.where(col("Country") == "United Kingdom").show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



### Dice Operation - Select a sub-cube from a datacube that contains items from United states and United Kingdom with quantities 0

In [27]:
dataframe.where((col("Country") == "United States") | (col("Country") == "United Kingdom")).where(col("Quantity") == 0).show()
dataframe.filter(((col("Country") == "United States") | (col("Country") == "United Kingdom") & (col("Quantity") == 0))).show()

# dataframe.filter("Country IN ('Norway', 'United Kingdom') AND Quantity > 0").orderBy("Country",ascending=True).show(100)
dataframe.filter("Country IN ('Norway', 'United Kingdom') AND Quantity > 0").sort("Country",ascending=True).show(100)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
| 

In [28]:
dataframe.where(((col("Country") == "United States") | (col("Country") == "United Kingdom")) & (col("Quantity") == 0)).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



### Which of the following codes peforms a rollup from a datacube?

In [29]:
dataframe.select("StockCode", "Country", (col("Quantity") > 0).alias("positive quantity")).show()

+---------+--------------+-----------------+
|StockCode|       Country|positive quantity|
+---------+--------------+-----------------+
|   85123A|United Kingdom|             true|
|    71053|United Kingdom|             true|
|   84406B|United Kingdom|             true|
|   84029G|United Kingdom|             true|
|   84029E|United Kingdom|             true|
|    22752|United Kingdom|             true|
|    21730|United Kingdom|             true|
|    22633|United Kingdom|             true|
|    22632|United Kingdom|             true|
|    84879|United Kingdom|             true|
|    22745|United Kingdom|             true|
|    22748|United Kingdom|             true|
|    22749|United Kingdom|             true|
|    22310|United Kingdom|             true|
|    84969|United Kingdom|             true|
|    22623|United Kingdom|             true|
|    22622|United Kingdom|             true|
|    21754|United Kingdom|             true|
|    21755|United Kingdom|             true|
|    21777

In [30]:
dataframe.where(col("Quantity") > 0).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S